In [1]:
from nhanes_data.nhanes_data_api import NHANESDataAPI

# Initialize the NHANESDataAPI object
nhanes_api = NHANESDataAPI()

In [3]:
NHANESDataAPI(data_directory="/workspaces/Final-Year-Project/data/NHANES")